In [54]:
from __future__ import print_function

First we load the functions and libraries necessary for this report.

In [68]:
run analysis_functions.ipynb #import all helper functions

In [16]:
# Import main dataset
df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
df = df.drop(['Unnamed: 0'], axis=1)

The structure of the analysis will be as such: 
1. Objective
2. Prepare data
3. Model Fitting, Tuning, and Evaluation
    - Benchmarks
        - Most Popular
        - ALS Matrix Factorization
    - LightFM
        - Vanilla FM / BPR
        - FM with User/Item Side Information
        - Parameter Tuning
4. Model Exploration
    - Metrics Used (NDCG, Recall, Precision, Coverage)
    - Performance of each Model (Table)
    - NDCG Metric by User Type 
        - Active/Non-Active (Aggregate Plays)
        - Diverse/Non-Diverse (Top 1000 Artists)
        - "Basic"/Non-"Basic" (Popular)
    - Scale:
        - NDCG by Size
        - Training Time / Predict Time
6. Conclusion / Next Steps

# 1. Objective

BLALBALBALBLA

# 2. Prepare Data

### 2a. Create Sparse Matrix from Dataset

In [17]:
#create sparse matrix
plays_sparse = create_sparse_matrix(df).astype('float')
print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

Creating sparse matrix...


/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


('Matrix Sparsity:', 99.8965986346416)


### 2b. Split Data to Train/Test

Split data into train test set, maintaining that each user still has some interactions intact.

In [18]:
# Split data into training and test sets
train, test, user_count = split_train_test_per_user(plays_sparse, 3, 10)
print("Percentage of original data masked:", pct_masked(plays_sparse, train))
print("Users masked:", user_count)


('Percentage of original data masked:', 0.06548419166887459)
('Users masked:', 8980)


In [35]:
#train is item by user to accomodate implicit and baseline training
train

<47102x9000 sparse matrix of type '<type 'numpy.float64'>'
	with 438337 stored elements in Compressed Sparse Row format>

# 3. Model Fitting, Tuning, and Evaluation

- Note on evaluation: use metrics
- what autotune does

## 3a. Benchmarks

### Baseline

Baseline recommends the most-popular artists to everyone.

In [36]:
model_baseline = Baseline(n_recs = 20)
model_baseline.fit(train)

Fitting baseline...
[3.17738e+05 3.48600e+04 6.91220e+04 ... 2.80000e+01 2.70000e+01
 2.40000e+01]


No tuning is necessary since no parameters. We then evaluate the test set below: 

In [46]:
coverage, precision, recall, ndcg = evaluate(model_baseline, "baseline", test, plays_sparse)
print("Precision:",precision*100,'%')
print("Recall:",recall*100,'%')
print("Coverage:",coverage*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

Evaluating model...



Precision: 1.1425389755 %
Recall: 7.61692650334 %
Coverage: 0.042461041994 %
Average NDCG per User: 4.533232513791051 %


### Model-Based (ALS)

Here we fit the model-based ALS Matrix Factorization using the implicit package from Homework 2 and use the parameters that were found to be optimized in the HW 2 report. 

In [86]:
model_als = implicit.als.AlternatingLeastSquares(factors = 30, regularization = 0.01)

# Train model
print("Fitting model...")
model_als.fit(train)

  0%|          | 0/15 [00:00<?, ?it/s]

Fitting model...


100%|██████████| 15.0/15 [00:06<00:00,  2.48it/s]


In [89]:
coverage, precision, recall, ndcg = evaluate(model_als, "implicit", test, plays_sparse)
print("Precision:",precision*100,'%')
print("Recall:",recall*100,'%')
print("Coverage:",coverage*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

Evaluating model...
implicit


Precision: 3.20100222717 %
Recall: 21.3400148478 %
Coverage: 8.54316164919 %
Average NDCG per User: 11.640612866692477 %


### LightFM (without side information)

Here we fit the LightFM model.

In [130]:
from lightfm import LightFM

model_fm_vanilla = LightFM(loss='bpr')

# Train Model
print("Fitting model...")
model_fm_vanilla.fit(train.T.tocsr(),user_features = None, item_features = None)

Fitting model...


Now we try to find the best hyperparameter for this model and use the model with the best hyperparameter to get our results.

In [93]:
run analysis_functions.ipynb #import all helper functions

In [131]:
coverage, precision, recall, ndcg = evaluate(model_fm_vanilla, "lightfm", test, plays_sparse)
print("Precision:",precision*100,'%')
print("Recall:",recall*100,'%')
print("Coverage:",coverage*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

Evaluating model...
lightfm


[   83  4832   331  1684  4644   999  5898  6842  3430   149  3576 16000
  2768  2632  8133  6108  7209  5536  1382    71]
[   83  4832   331   149  1684  4644  5898  3430   999  3576  2768 16000
  6842  8133  2632  6108  1382   619    71  4326]
[   83  4832   331  1684  4644  2768  5898  3576  3430   149  6842  2632
   999  6108 16000  8133  3536  1414  1382    71]
[   83   331  4832  1684   149  3430  4644  5898  3576   999  2768 16000
  8133  6842  6108  2632  1382   619  4326    71]
[   83   331  4832  1684   149  3576  4644  3430  2768  5898   999  6842
  8133  2632 16000  6108  4326  1382  1414   619]
[   83  4832   331  1684  4644  2768  3430  5898  6842  3576  2632   999
  6108 16000   149  1414  3536  8133    71  1382]
[   83  4832   331  1684  4644  2768  3430  5898  6842  3576   149 16000
  2632  6108  1414   999  1382  8133  3536   619]
[   83   331  4832  1684   149  4644  2768  3430  3576  5898   999  6842
  8133 16000  2632  6108  1414  1382  3536  7209]
[   83  4832   3

[   83  4832   331  1684  4644  5898   149  3576  2768  6842   999  8133
  3430  2632  6108 16000  1382  1414  3536  4326]
[   83  4832   331  1684  4644  2768  3430  5898  6842  3576   149  2632
 16000   999  3536  1414  6108  1382  8133    71]
[   83   331  4832  1684   149  5898  4644  2768   999  3576  3430  8133
  6842 16000  2632  6108  1382  4326  3536    71]
[   83  4832   331  1684  4644  5898   149  3430  2768  6842  3576   999
  8133  1382  6108 16000  1414  2632  5536   619]
[   83  4832   331  1684  4644  3576   149  5898  2768  6842  3430  8133
   999  6108  2632 16000  1414  1382   619  5536]
[   83  4832   331  1684  4644  5898  2768  3430   149  2632  3576 16000
  6108   999  6842  8133  1414  4326  1382  3536]
[   83  4832   331  1684  4644  5898  3430   149  2768  6842   999  3576
 16000  2632  1382  8133  6108  1414  3536  7209]
[   83  4832   331  1684  4644   149  3430  5898  3576   999  6842  2768
  8133  6108 16000  2632  1382   619  5536  1414]
[   83  4832   3

[   83  4832   331  1684  4644  2768  5898  3576  6842  2632  3430   999
 16000   149  8133    71  6108  3536  1414  3836]
[   83  4832   331  1684  4644  5898   149  3430   999  6842  3576 16000
  8133  2768  6108  2632  1382   619  5536    71]
[   83  4832   331  1684  4644  3430  5898  2768   149   999  6842  3576
  6108 16000  1382  8133  2632   619  3142    71]
[   83   331  4832  1684  5898  4644   999   149  2768  3576  8133  6842
  3430  2632  6108 16000  4326  7209  1382  5536]
[   83  4832   331  1684  4644  5898   999   149  3430  6842 16000  8133
  2768  6108  3576  1382  2632  5536  3142  7209]
[   83   331  4832  1684  4644  2768   149  6842  3576  5898  3430  8133
   999  6108  1382  2632 16000  1414  5536  3536]
[   83  4832   331  1684  4644  5898  2768  3576  2632  6842  3430   149
   999  6108 16000  8133  1414  3536  3836  1382]
[   83  4832   331  1684   149  3576  5898  2768  3430  4644  6842  8133
  2632   999 16000  6108  1382  3536  1414  4326]
[   83  4832   3

[   83  4832   331  1684  4644  2768  5898  6842  3576  2632  3430  1414
   999   149 16000  8133  3536  6108    71   619]
[   83  4832   331  1684  3430  4644   149  2768  5898   999  3576  6108
 16000  6842  1382  8133  2632  1414   619  3536]
[   83  4832   331  1684  4644  5898  2768  6842  3430  3576  2632  6108
   999 16000  8133   149  1414  1382  3536    71]
[   83  4832   331  1684  4644  5898   149   999  6842  3576  3430  2768
  8133 16000  2632  6108  1382  5536   619  7209]
[   83   331  4832  1684   149  3430  2768  3576  4644  5898  6842  6108
  8133 16000  2632  1414  1382   619   999  3536]
[   83   331  4832  1684  5898   149  4644  2768  3430  3576   999  8133
  6842  6108 16000  2632  1382  4326    71  3536]
[   83   331  4832   149  3430  1684  2768  3576  4644  5898  6842  8133
  1382  6108   999 16000  2632  1414  3536  3142]
[   83  4832   331  1684  4644  2768  3430  5898  6842  3576   149  2632
   999 16000  6108  8133  1414  1382  3536    71]
[   83  4832   3

...

[   83  4832   331  1684  4644  3576  5898  6842  2768  3430  2632   149
  8133   999  6108 16000  1414  3536   619  3836]
[   83  4832   331  1684  3430  4644   149  2768  5898 16000  3576  6842
   999  6108  1382  8133  2632    71  1414  3536]
[   83  4832   331  1684  4644  5898  2768  3430   999  6108   149  3576
 16000  6842  8133   619  1382  2632  1414    71]
[   83  4832   331  1684  4644  5898  3576  2768  3430  6842   149  8133
   999  2632 16000  6108  4326    71   619  3836]
[   83   331  4832  1684  2768  5898  4644   149  3430  3576  2632   999
  6842 16000  8133  6108  3536  1414  4326  7209]
[   83   331  4832  1684   149  5898  2768  4644  3430  3576   999  8133
  6842  6108 16000  2632  1382  4326    71  3536]
[   83  4832   331  1684  4644  5898  2768  6842  3576  2632  3430   999
   149  6108 16000  8133  1414  3536  1382    71]
[   83  4832   331  1684  4644  3576  5898   149  6842  3430  2768  8133
 16000   999  2632  6108  1382   619    71  3836]
[   83  4832   3

[   83  4832   331  4644  1684  5898  2768  6842  3576  2632  3430   999
  6108 16000  1414  8133   149  3536  1382    71]
[   83  4832   331  1684  4644  2768  3430  5898  3576  6842  6108  2632
  8133 16000   999   149  1414  1382   619    71]
[   83   331  4832   149  1684  2768  3576  5898  3430  4644  8133  6842
   999  6108  2632 16000  1382  4326  1414  3536]
[   83  4832   331  1684   149  3430  4644  2768  3576  5898 16000   999
  6842  6108   619  1382  8133  1414  2632  3536]
[   83   331  4832  1684   149  4644  5898   999  8133  3576  6842  3430
  2768  6108  1382 16000  2632  5536  7209  4326]
[   83  4832   331  1684  4644  2768  3576  5898  3430  2632   999  6842
   149  6108  8133 16000  4326  1414  3836   619]
[   83  4832   331  1684  4644  2768  5898  3430  6842  3576  2632  6108
 16000   999  1414   149  8133  3536  1382    71]
[   83  4832   331  1684  3576  2768  4644   149  6842  3430  5898  8133
  2632  1414   999  6108  3536 16000  1382  5536]
[   83  4832   3

[   83  4832   331  1684  4644  2768  5898  3430   149  3576  6108  6842
  8133   999  2632 16000  1382  1414   619  3536]
[   83  4832   331  1684  4644  2768  3430  5898  6108  6842  3576   999
 16000  1414  2632   149  8133  1382  7209   619]
[   83  4832   331  1684  4644   999  5898  3576  3430   149  2768  6842
  8133  6108 16000  2632  1382  5536   619  7209]
[   83  4832   331  1684  4644   149   999  3430  5898  2768 16000  3576
  6842  8133  2632  6108  7209   619  1382    71]
[   83  4832   331  1684  4644   149  2768  5898  3430  3576   999  6842
 16000  2632  8133  6108   619  1382  3536    71]
[   83  4832   331  1684   149  5898  3576  4644  2768  6842  8133  3430
  2632   999 16000  6108  4326  1382  3536    71]
[   83   331  4832  1684  2768   149  4644  3430  5898  3576   999  6842
  2632 16000  6108  8133  3536  1414  1382  7209]
[   83  4832   331  1684  4644  2768  3430  3576   149  6842   999  5898
 16000  6108   619  8133  2632  1414  3536    71]
[   83  4832   3

[   83  4832   331  4644  1684  5898  6842  3430  2768  3576   149  2632
  8133   999  6108  1382 16000  1414  3536  5536]
[   83   331  4832  1684  2768  4644  3576  6842   149  5898  3430  2632
  8133   999  3536 16000  1414    71  6108  1382]
[   83  4832   331  1684  4644  2768  3576  3430  5898   149  6842  6108
  2632  8133 16000   999  1414   619  3536  3836]
[   83  4832   331  1684  3576  5898  4644   149  2768  3430  6842  2632
  8133  6108 16000   999  1414  4326  3836  3536]
[   83  4832   331  1684  2768  4644  3430  5898  3576  2632  6842   149
   999 16000  6108  3536  1414  8133    71  7209]
[   83  4832   331  1684  4644  5898   999  2768   149  6842  3576  3430
  8133  2632  6108 16000  7209  1382  5536  1414]
[   83  4832   331  1684  4644  2768  3576  6842  3430  5898  2632  8133
   149  1414  6108 16000  3536   999  1382   619]
[   83   331  4832  1684   149  3430  4644  2768  5898   999  3576  6108
 16000  8133  6842  1382   619  2632  3142  1414]
[   83  4832   3

[   83  4832   331  4644  5898  1684  6842  2768  3576  2632  8133  3430
   999   149  6108  1382  1414 16000  3536    71]
[   83   331  4832  1684  4644  2768  5898  6842  3576  3430   999   149
  8133  2632  6108  1414 16000  1382  7209  3536]
[   83  4832   331  4644  2768  1684  3430  3576  5898  6842   149  6108
  1382  8133 16000  1414  2632  3536   999   619]
[   83  4832   331  1684  4644   149  3430  5898  2768  3576  6842   999
  8133  1382 16000  6108  2632  1414  3536   619]
[   83  4832   331   149  1684  4644  6842  5898  3576  8133  3430   999
  2768  1382  2632  6108 16000  5536  1414   619]
[   83   331  4832  1684  3430   149  3576  2768  4644  5898  8133  6842
  6108 16000   999  2632  1382  1414  4326   619]
[   83  4832   331  1684  4644  2768  3576  5898  6842   999  3430  2632
  8133  6108   149 16000    71  4326  1414  7209]
[   83  4832   331  1684  5898   149  2768  4644  3430  3576  6842 16000
   999  2632  6108  8133  1382  3536    71  1414]
[   83  4832   3

### LightFM (with side information)

In [ ]:
#insert side info here here
user_features = None
item_features = None

model_fm_features = LightFM(no_components=30, loss='warp')

# Train Model
print("Fitting model...")
model.fit(train.T.tocsr(),user_features = None, item_features = None)

Now we try to find the best hyperparameter for thi smodel with the best hyperparameter to get our results.

### Summary of Performance Results (Table)

# 4. Model Exploration

Next, we explore these models even more. How do they perform in regards to size, different user population?

## 4a. Performance by User Type

### Active/Non-Active

### Diverse/Non-Diverse

### Basic/Non-Basic

## 4b. Performance by Input Size 

### Accuracy/NDCG

### Training/Predict Time

# 5. Conclusion / Next Steps

# OLD STUFF: 

# Cross Validation and Parameter Tuning (k-fold)

In this section, we use k-fold cross validation to tune hyperparameters and evaluate our models.

### Splitting into test and training sets

In [16]:
# Cross Validation test
k=5
train_list, test_list, user_count = split_train_test_per_user(plays_sparse,k,20,cross_valid=True)

## Evaluate and tune ALS

Tuning two parameters: number of latent factors and the regularization factor. 

For latent factors, we try values [10,20,30,40,50,60]

For regularization factors, we try [.01,.03,.05,.07]

In [ ]:
start = time.time()
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)
stop = time.time()
total = stop-start

In [ ]:
# Plot heatmap of parameter tuning results
sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], 
            xticklabels=['0.01','0.03','0.05','0.07'], 
            yticklabels=['10','20','30','40','50','60'], 
            cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

Here we analyze the scalability of ALS by looking at datasets with 9k, 20k, 60k, and 150k users:

In [ ]:
# The following block imports larger datasets for scaling tests, these expanded CSVs are not available in the repo
files9k = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
files20k = pd.read_csv('lastfm_20k_users.csv', na_filter=False)
files60k = pd.read_csv('lastfm_60k_users.csv', na_filter=False)
files150k = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
files40k = get_users(files150k, 40000)
files = [files9k, files20k, files40k, files60k]

In [ ]:
# Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
size = [9000, 20000, 40000, 60000]
ndcg_size = []
recall_size = []
for i in files:
    model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)

    #create sparse matrix
    plays_sparse = create_sparse_matrix(i).astype('float')
    print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

    train, test, user_count = split_train_test_per_user(plays_sparse, 4, 20)

    # train model 
    print("Fitting model...")
    model.fit(train, show_progress=True)

    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size.append(recall)
    ndcg_size.append(ndcg)

In [ ]:
# Plot scalability of ALS model
ndcg_size_df = pd.DataFrame({'N':size,
                       'NDCG': ndcg_size})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_size_df)
plt.title('NDCG by Input Size for ALS')
plt.xlabel('Number Of Users')
plt.show()

Here we analyze the catalog coverage of the ALS model:

In [ ]:
# Calculate catalog coverage of ALS model with optimal parameters
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
model.fit(plays_sparse)
users = list(df.user_id.unique())
catalog = []
for i in range(0,len(users)):
    for x,y in model.recommend(i,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
        if x not in catalog:
            catalog.append(x)
print('Catalog Coverage is', len(catalog)/plays_sparse.shape[1])

# Tune ALS

Below we will tune the hyperparameters of ALS from a given range of hyperparameters.

In [ ]:
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)

sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], xticklabels=['0.01','0.03','0.05','0.07'], yticklabels=['10','20','30','40','50','60'], cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

# Input Size vs Training Time

Below you can run the code using a larger dataset to evaluate performance tiem compared with input tiem

In [ ]:
#helper function to grab needed users
def get_users(df, n):
    sample_userid = df["user_id"].unique()
    sample_userid = np.random.choice(sample_userid, size = n, replace = False)

    #grab rows with sample user id
    df_sample = df[df.user_id.isin(sample_userid)].reset_index(drop = True)

    return df_sample

#in order to run this analysis, you need to download the 150k data at: 
#https://www.dropbox.com/s/qd8rnlxsuq0rjll/last_fm_bigger_data.zip?dl=0

#read in large dataset
df_150 = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
df_150 = df_150.drop(['Unnamed: 0'], axis=1)

# ALS

In [ ]:
#plot ALS training time vs input size
#calculate training time for different input sizes for ALS
import sns
import time

size = [9000,20000,40000,60000,80000]
train_time = list()

#get train data for each input size
for users in size:
    df = get_users(df_150, users)
    plays_sparse = create_sparse_matrix(df)

    # K-Nearest Neighbors
    model = implicit.als.AlternatingLeastSquares(50)
    
    start = time.time()
    # train model 
    print("Fitting model...")
    model.fit(plays_sparse, show_progress=True)
    stop = time.time()
    total = stop-start
    train_time.append(total)

#plot
df = pd.DataFrame({'n':size,'time':train_time})
sns.pointplot(x="n",y="time",data=df)
plt.title('Training Time by Input Size for KNN')
plt.xlabel('Number of Users')
plt.ylabel('Time (s)')